### Import Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical

### Data Loading

***Loading test-set***

In [ ]:
base_dir_test="input/Test"

In [ ]:
class_labels=os.listdir(base_dir_test)
print(class_labels)

In [ ]:
test_data=[]
count=0

for label in class_labels:
    i=0
    path = os.path.join(base_dir_test, label)
    print(path)
    for img in os.listdir(path):
        try:
            image=load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(256,256))
            image=img_to_array(image)
            image=image/255.0
            test_data.append([image,count])
        except Exception as e:
                pass
    count=count+1

In [ ]:
print(len(test_data))

In [ ]:
x_test,Y_test =zip(*test_data)

In [ ]:
x_test

In [ ]:
print(Y_test)

In [ ]:
X_test=np.array(x_test)
y_test=np.array(Y_test)

In [ ]:
print(X_test.shape,y_test.shape)

In [ ]:
from random import randint
def show_images(X,y):
    labels =class_labels
    x,y = X,y
    plt.figure(figsize=(15, 15))
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        idx = randint(0, X_test.shape[0]-1)
        plt.imshow(x[idx])
        plt.axis("off")
        plt.title("Class:{}".format(labels[y[idx]]))
show_images(X_test,y_test)

***Loading train-set***

In [ ]:
base_dir_train="input/Train"

In [ ]:
class_labels=os.listdir(base_dir_train)
print(class_labels)

In [ ]:
train_data=[]
count=0

for label in class_labels:
    i=0
    path = os.path.join(base_dir_train, label)
    print(path)
    for img in os.listdir(path):
        try:
            image=load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(256,256))
            image=img_to_array(image)
            image=image/255.0
            train_data.append([image,count])
        except Exception as e:
                pass
    count=count+1

In [ ]:
print(len(train_data))

In [ ]:
x_train,Y_train =zip(*train_data)

In [ ]:
x_train

In [ ]:
print(Y_train)

In [ ]:
X_train=np.array(x_train)
y_train=np.array(Y_train)

In [ ]:
print(X_train.shape,y_train.shape)

In [ ]:
from random import randint
def show_images(X,y):
    labels =class_labels
    x,y = X,y
    plt.figure(figsize=(15, 15))
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        idx = randint(0, X_train.shape[0]-1)
        plt.imshow(x[idx])
        plt.axis("off")
        plt.title("Class:{}".format(labels[y[idx]]))
show_images(X_train,y_train)

### Algorithm - Convolutional Neural Network

In [ ]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
cnn_model=Sequential()

cnn_model.add(Conv2D(filters=64,kernel_size=3,strides=(1,1),padding="same",activation="relu",input_shape = (256,256,3)))
cnn_model.add(MaxPool2D(pool_size=(3,3),padding="same"))

cnn_model.add(Conv2D(filters=128,kernel_size=3,strides=(1,1),padding="same",activation="relu"))
cnn_model.add(MaxPool2D(pool_size=(3,3),padding="same"))

cnn_model.add(Conv2D(filters=128,kernel_size=3,strides=(1,1),padding="same",activation="relu"))
cnn_model.add(MaxPool2D(pool_size=(3,3),padding="same"))

cnn_model.add(Conv2D(filters=256,kernel_size=3,strides=(1,1),padding="same",activation="relu"))
cnn_model.add(MaxPool2D(pool_size=(3,3),padding="same"))

cnn_model.add(Flatten())
cnn_model.add(Dropout(rate=0.4))
cnn_model.add(Dense(units=512,activation="relu"))
cnn_model.add(Dense(units=len(class_labels),activation="softmax"))

In [ ]:
cnn_model.compile(optimizer=Adam(learning_rate=1e-3,decay=1e-4),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
cnn_model.summary()

In [ ]:
history=cnn_model.fit(x=X_train,y=y_train,batch_size=64,epochs=10,validation_data=(X_test,y_test))

### Accuracy and Loss plot-graphs

In [ ]:
plt.figure(figsize=(18,8))
plt.rcParams["font.size"]=15
plt.plot(history.history["accuracy"],label="accuracy")
plt.plot(history.history["val_accuracy"],label="val_accuracy")
plt.title(label="Accuracy and Val_accuracy plot-graphs")
plt.legend()
plt.show()

plt.figure(figsize=(18,8))
plt.plot(history.history["loss"],label="loss")
plt.plot(history.history["val_loss"],label="val_loss")
plt.title(label="Loss and Val_loss plot-graphs")
plt.legend()
plt.show()

In [ ]:
cnn_prediction=cnn_model.predict(X_test,batch_size=64,verbose=1)

In [ ]:
print(cnn_prediction)

In [ ]:
cnn_labels=[]
for i in range(len(cnn_prediction)):
    cnn_labels.append(np.argmax(cnn_prediction[i]))

In [ ]:
print(cnn_labels)

In [ ]:
true_labels=[]
for i in range(len(y_test)):
    true_labels.append(np.argmax(y_test[i]))

In [ ]:
print(true_labels)

### Result Analysis

***Accuracy Score***

In [ ]:
cnn_accuracy=accuracy_score(y_true=true_labels,y_pred=cnn_labels)
print("CNN validated accuracy is {:.2f}%".format(cnn_accuracy*100.0))

***Classification Report***

In [ ]:
print(classification_report(y_true=true_labels,y_pred=cnn_labels,target_names=class_labels))

***Confusion Matrix***

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
ax=plot_confusion_matrix(conf_mat=confusion_matrix(y_true=true_labels,y_pred=cnn_labels),
                        figsize=(10,5),
                        class_names=class_labels,
                        cmap=plt.cm.Greens)
plt.title(label="CNN Confusion Matrix")
plt.xticks(rotation=90)
plt.show()

***Model Saving***

In [ ]:
cnn_model.save(filepath="models/Convolutional_Neural_Network.h5")